<a href="https://colab.research.google.com/github/cpython-projects/python_da_06_11_25/blob/main/lesson_15_hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🏡 Сервіс порівняння цін на будинки

Ви працюєте аналітиком у стартапі, який запускає сервіс для порівняння цін на будинки. Сервіс має допомагати користувачам:

* дізнатися, чи адекватна ціна конкретного будинку;
* порівнювати схожі будинки між собою;
* виділяти масовий і преміальний сегменти ринку.

**Датасет:** [House Sales in King County, USA (Kaggle)](https://www.kaggle.com/datasets/harlfoxem/housesalesprediction)
Файл: `kc_house_data.csv`

## **Завдання 1. Типова ціна масового ринку**

Яку **типову ціну** сервіс має показувати користувачу для орієнтиру на масовому ринку житла?

(Яке значення краще відображає «нормальну» ціну більшості будинків і чому?)

## **Завдання 2. Аномалії та преміальний сегмент**

1. Які будинки можна віднести до **преміального сегменту**?
2. Які будинки виглядають **занадто дорогими або занадто дешевими**?
3. Чому такі будинки **не варто використовувати** для оцінки масового ринку?

## **Завдання 3. Порівняння будинків різного розміру**

Як коректно **порівнювати між собою будинки різної площі**, щоб оцінка ціни була справедливою?

(Який показник варто використати замість “просто ціни”?)

## **Завдання 4. Сегментація ринку за ціною**

Як можна розділити ринок будинків на **цінові сегменти**?

* Скільки сегментів ви б виділили?
* Чим вони відрізняються між собою за ціною?